In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import scipy.io
import pandas as pd
from cycler import cycler
from sklearn import metrics

plt.rcParams['font.size'] = 7

### Load Data

In [ ]:
# Input parameters
T = 25                                              # Period of measurement (ns)

# Monte Carlo parameters
taus = np.arange(0.2, 15, 0.4)                      #Lifetimes (ns)
tns = 2**(np.arange(9)+2)                           #Time-bins
irfSigs = np.array((0.01, 0.1, 0.25, 0.5, 1, 2))    #IRF Standard Deviations (ns)
iterats = 5000                                      #Repetitions of simulations
numPhots = 75000                                    #Number of photons per measurement

# Load MC data
F_dirac = np.load('F_dirac.npy')
F_known = np.load('F_known.npy')
F_unknown = np.load('F_unknown.npy')
F_knownsq = np.load('F_knownsq.npy')

np.set_printoptions(precision=2)
print("Monte Carlo Simulation Parameters\n")
print("Taus(ns):\n{}\n".format(taus))
print("Time Bins:\n{}\n".format(tns))
print("IRF Sigma (ns):\n{}\n\n".format(irfSigs))


# Fisher parameters
TauN = 151 # The workspace stores TauN 2D matrices named 'Expression1', 'Expression2', ... (each matric is related to a simulated lifetime
BinN = 12  # The first dimension of each matrix relates to simulated bin numbers
SigN = 7   # The second dimension is the number of IRF widths simulated

Sig0 = np.array((0.01, 0.1, 0.25, 0.5, 1, 2, 5))   # Simulated IRF widths (std dev)
Tau0 = 0.01*1.05**(np.arange(TauN))                # Simulated Taus
Bin0 = 2**(np.arange(BinN)+1)                      # Simulated bin numbers

# Load Fisher data
mathematicadata = scipy.io.loadmat('GaussianForMatlab.mat')

if TauN == len(mathematicadata):
    fisherdata = np.zeros((TauN, BinN, SigN))      # 3D array for variance from Fisher information analysis [tau, tn, IRF]
    for TauI in range(TauN):
        fisherdata[TauI,:,:] = mathematicadata['Expression'+str(TauI+1)]
    fisherdata[fisherdata==100] = np.nan           # Added to handle possible numerical instabilities in Mathematica
    fisherdata[fisherdata<1e-5] = np.nan
else:
    print('TauN not equal to length of Mathematica data')

print("Fisher Information Analysis Parameters\n")
print("Taus(ns):\n{}\n".format(Tau0))
print("Time Bins:\n{}\n".format(Bin0))
print("IRF Sigma (ns):\n{}\n".format(Sig0))
print("Fisher Data shape:\n {}\n".format(fisherdata.shape))

In [ ]:
def kollner(tau, T, tn):
    """Evaluates variance of measurement from eqn in Kollner and Wolfrum (1992)"""
    r = T/tau;
    v = np.zeros((tau.size, tn.size))
    v = ((tn/r)**2)*(1-np.exp(-r))*(((np.exp(r/tn)*(1-np.exp(-r)))/((np.exp(r/tn)-1)**2))-((tn**2)/(np.exp(r)-1)))**-1
    return v

### Figure 1: Fisher information

In [ ]:
# Figure 1a-e with inset
# Fisher and Kollner
# IRF 0.1, 0.25, 0.5, 1, 2ns
# Bin 4, 16, 64, 256, 1024

# Determine index for parameters to plot
plotirf = np.array((0.1, 0.25, 0.5, 1, 2))
plottn = np.array((4, 16, 64, 256, 1024))

irf_idx = np.in1d(Sig0, plotirf)
irf_idx = np.where(irf_idx==True)[0]

tn_idx = np.in1d(Bin0, plottn)
tn_idx = np.where(tn_idx==True)[0]

# Generate plots
for n in tn_idx:
    fig, ax = plt.subplots(figsize=[2, 2])
    ax.plot((-1, 16), (1,1), color='gray')
    ax.plot(Tau0, np.squeeze(fisherdata[:,n, irf_idx]**(1/2)))
    ax.plot(Tau0, kollner(Tau0, T, Bin0[n])**(1/2), 'k--')
    plt.ylim(0,3)
    plt.xlim(-1,16)
    plt.yticks(range(4))
    plt.xticks((0,5,10,15))
    plt.title(str(Bin0[n]) + ' time-bins')
    plt.ylabel('F-value')
    plt.xlabel('lifetime (ns)')
    #plt.legend(Sig0[irf_idx])
    
    axins = ax.inset_axes([7.4, 0.15, 8, 1], transform=ax.transData)
    axins.plot((-1, 16), (1,1), color='gray')
    axins.plot(Tau0, np.squeeze(fisherdata[:,n, irf_idx]**(1/2)))
    axins.plot(Tau0, kollner(Tau0, T, Bin0[n])**(1/2), 'k--')
    x1, x2, y1, y2 = -0.25, 3.75, 0.95, 1.45
    axins.set_yticks(range(4))
    axins.set_xticks((0,5,10,15))
    axins.set_xlim(x1, x2)
    axins.set_ylim(y1, y2)
    axins.set_xticklabels('')
    axins.set_yticklabels('')
    ax.indicate_inset_zoom(axins)
    plt.show
#    plt.savefig('F_DiracFisher_Bins_'+str(Bin0[n])+'.pdf')

In [ ]:
def kollner2d(tau, T, tn):
    """Evaluates variance of measurement from eqn in Kollner and Wolfrum (1992). Input multiple tn. Ouput 2D"""
    r = T/tau;
    v = np.zeros((tau.size, tn.size))
    v = ((tn[np.newaxis,:]/r[:,np.newaxis])**2)*(1-np.exp(-r[:,np.newaxis]))*(((np.exp(r[:,np.newaxis]/tn[np.newaxis,:])*(1-np.exp(-r[:,np.newaxis])))/((np.exp(r[:,np.newaxis]/tn[np.newaxis,:])-1)**2))-((tn[np.newaxis,:]**2)/(np.exp(r[:,np.newaxis])-1)))**-1
    return v

In [ ]:
# Figure 1f
# Loss of Gaussian vs Dirac

# Determine index for parameters to plot
plottn = np.array((4, 16, 64, 256, 1024))
tn_idx = np.in1d(Bin0, plottn)
tn_idx = np.where(tn_idx==True)[0]

# Determine standard deviation for Dirac Case
kdata = kollner2d(Tau0, T, Bin0)**(1/2)

# Calculate loss between Gaussian and Dirac
lossdata = fisherdata**(1/2) - kdata[:,:,np.newaxis]
lossdata[lossdata<0] = 0
L = np.zeros((Bin0.size, Sig0.size))

for b in range(Bin0.size):
    for s in range(Sig0.size):
        L[b,s] = metrics.auc(Tau0[:], lossdata[:,b,s]) #Note: For loss data AUC from 0.6ns to 15

# Generate plots
custom_cycler = (cycler(color=['#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']))
fig, ax = plt.subplots(figsize=[2, 2])
ax.set_prop_cycle(custom_cycler)
ax.plot(Sig0[1:], np.transpose(L)[1:,tn_idx])
plt.ylabel('loss (ns)')
plt.xlabel('IRF (ns)')
plt.show
plt.legend(Bin0[tn_idx])
plt.title('Gauss. IRF - Dirac IRF')
#plt.savefig('FisherLoss.pdf')

### Figure 2: Monte Carlo simulations

In [ ]:
# Figure 2
# Monte Carlo and Dirac
# IRF 0.1, 0.25, 0.5, 1, 2ns
# Bin 16, 64, 256

# Determine index for parameters to plot
plottn = np.array((16, 64, 256))

irf_idx_mc = np.in1d(irfSigs, plotirf)
irf_idx_mc = np.where(irf_idx_mc==True)[0]

tn_idx_mc = np.in1d(tns, plottn)
tn_idx_mc = np.where(tn_idx_mc==True)[0]

# Generate plots
for n in range(plottn.size):
    fig, ax = plt.subplots(figsize=[2, 2])
    ax.plot((-1, 16), (1,1), color='gray')
    ax.plot(taus, np.squeeze(F_known[:,tn_idx_mc[n], irf_idx_mc]))
    ax.plot(Tau0, kollner(Tau0, T, plottn[n])**(1/2), 'k--')
    plt.ylim(0,3)
    plt.xlim(-1,16)
    plt.yticks(range(4))
    plt.xticks((0,5,10,15))
    plt.title(str(plottn[n]) + ' time-bins')
    plt.ylabel('F-value')
    plt.xlabel('lifetime (ns)')
    #plt.legend(Sig0[irf_idx])
    
    axins = ax.inset_axes([7.4, 0.15, 8, 1], transform=ax.transData)
    axins.plot((-1, 16), (1,1), color='gray')
    axins.plot(taus, np.squeeze(F_known[:,tn_idx_mc[n], irf_idx_mc]))
    axins.plot(Tau0, kollner(Tau0, T, plottn[n])**(1/2), 'k--')
    x1, x2, y1, y2 = -0.25, 3.75, 0.95, 1.45
    axins.set_yticks(range(4))
    axins.set_xticks((0,5,10,15))
    axins.set_xlim(x1, x2)
    axins.set_ylim(y1, y2)
    axins.set_xticklabels('')
    axins.set_yticklabels('')
    ax.indicate_inset_zoom(axins)
    plt.show
    #plt.savefig('F_FisherMC_Bins_'+str(plottn[n])+'.pdf')

### Figure 3: IRF Shape

In [ ]:
# Fig 3a
# Plot comparing IRF shapes

T = 25
tn = 256
ti = np.arange(tn)+1
twidth = T/tn
t = ti*twidth
t0=int(tn/2)

irfSig = np.array((0.1, 1, 2))

def irfgaus(x):
    normfact = sum((1/np.sqrt(2*math.pi*(irfSig[np.newaxis,:]/twidth)**2))*np.exp(-(x[:,np.newaxis]-t0)**2/(2*(irfSig[np.newaxis,:]/twidth)**2)))
    return (1/np.sqrt(2*math.pi*(irfSig[np.newaxis,:]/twidth)**2))*np.exp(-(x[:,np.newaxis]-t0)**2/(2*(irfSig[np.newaxis,:]/twidth)**2))/normfact

def irfsqr(x):
    normfact = sum(np.abs(x[:,np.newaxis]-t0)<=((2.355*irfSig[np.newaxis,:])/(2*twidth)))
    return (np.abs(x[:,np.newaxis]-t0)<=((2.355*irfSig[np.newaxis,:])/(2*twidth)))/normfact

irfs_g = irfgaus(ti)/np.max(irfgaus(ti),0)
irfs_s = irfsqr(ti)/np.max(irfsqr(ti),0)

custom_cycler = (cycler(color=['#1f77b4', '#d62728', '#9467bd']))
fig, ax = plt.subplots(figsize=[7.5, 2])
ax.set_prop_cycle(custom_cycler)
ax.plot(t-T/2, irfs_g, '.-', markersize=3)
ax.set_prop_cycle(custom_cycler)
ax.plot(t-T/2, irfs_s, '.:', markersize=3)
plt.yticks(np.linspace(0,1,3))
#plt.legend(('0.1 Gauss', '1.0 Gauss', '2.0 Gauss', '0.1 Square', '1.0 Square', '2.0 Square'))
plt.xlabel('time (ns)')
plt.ylabel('normalized intensity')
plt.title('Gaussian and Rectangular IRFs')
plt.show
#plt.savefig("IRFs_nolegend.pdf")

In [ ]:
# Fig 3b-c
# Comparing Gauss to square IRF

# Determine index for parameters to plot
plotirf = np.array((0.1, 1, 2))
plottn = np.array((256))

irf_idx_mc = np.in1d(irfSigs, plotirf)
irf_idx_mc = np.where(irf_idx_mc==True)[0]

tn_idx_mc = np.in1d(tns, plottn)
tn_idx_mc = np.where(tn_idx_mc==True)[0]

# Generate plots
custom_cycler = (cycler(color=['#1f77b4', '#d62728', '#9467bd']))
fig, ax = plt.subplots(figsize=[2, 2])
ax.plot((-1, 16), (1,1), color='gray', label = '_nolegend_')
ax.set_prop_cycle(custom_cycler)
ax.plot(taus, np.squeeze(F_known[:,tn_idx_mc, irf_idx_mc]))
ax.set_prop_cycle(custom_cycler)
ax.plot(taus, np.squeeze(F_knownsq[:,tn_idx_mc, irf_idx_mc]), linestyle='dotted')
ax.plot(Tau0, kollner(Tau0, T, plottn)**(1/2), 'k--')
plt.ylim(0,3)
plt.xlim(-1, 16)
plt.yticks(range(4))
plt.title(str(plottn) + ' time-bins')
plt.ylabel('F-value')
plt.xlabel('lifetime (ns)')

axins = ax.inset_axes([7.4, 0.15, 8, 1], transform=ax.transData)
axins.plot((-1, 16), (1,1), color='gray')
axins.set_prop_cycle(custom_cycler)
axins.plot(taus, np.squeeze(F_known[:,tn_idx_mc, irf_idx_mc]))
axins.set_prop_cycle(custom_cycler)
axins.plot(taus, np.squeeze(F_knownsq[:,tn_idx_mc, irf_idx_mc]), linestyle='dotted')
axins.plot(Tau0, kollner(Tau0, T, plottn)**(1/2), 'k--')
x1, x2, y1, y2 = -0.25, 3.75, 0.95, 1.45
axins.set_yticks(range(4))
axins.set_xticks((0,5,10,15))
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticklabels('')
axins.set_yticklabels('')
ax.indicate_inset_zoom(axins)
#plt.legend(np.append(plotirf, 'square'))
plt.show
#plt.savefig('F_FisherMC_GaussSquare_' + str(plottn) + '.pdf')

In [ ]:
# Fig 3d
# Loss of Rectangular vs Gaussian

# Determine index for parameters to plot
plottn = np.array((16, 256))

tn_idx_mc = np.in1d(tns, plottn)
tn_idx_mc = np.where(tn_idx_mc==True)[0]

# Compute loss
lossdata = F_known - F_knownsq
L = np.zeros((tns.size, irfSigs.size))

for b in range(tns.size):
    for s in range(irfSigs.size):
        L[b,s] = metrics.auc(taus[1:], lossdata[1:,b,s])

# Generate plots
custom_cycler = (cycler(color=['#e377c2', '#bcbd22']))
fig, ax = plt.subplots(figsize=[2, 2])
ax.set_prop_cycle(custom_cycler)
ax.plot(irfSigs, np.transpose(L)[:,tn_idx_mc])
plt.ylabel('loss (ns)')
plt.xlabel('IRF (ns)')
plt.show
#plt.legend(tns[tn_idx_mc])
plt.title('Gauss. IRF - Rect. IRF')
#plt.savefig('SquareGaussLoss.pdf')

In [ ]:
# Fig 3e-f
# Comparing measured and fitted IRF

# Determine index for parameters to plot
plotirf = np.array((0.25))
plottn = np.array((256))

irf_idx_mc = np.in1d(irfSigs, plotirf)
irf_idx_mc = np.where(irf_idx_mc==True)[0]

tn_idx_mc = np.in1d(tns, plottn)
tn_idx_mc = np.where(tn_idx_mc==True)[0]

# Generate plots
fig, ax = plt.subplots(figsize=[2, 2])
ax.plot((-1, 16), (1,1), color='gray', label = '_nolegend_')
ax.plot(taus, np.squeeze(F_known[:,tn_idx_mc, irf_idx_mc]))
ax.plot(taus, np.squeeze(F_unknown[:,tn_idx_mc, irf_idx_mc]))
ax.plot(Tau0, kollner(Tau0, T, plottn)**(1/2), 'k--')
plt.ylim(0,3)
plt.xlim(-1, 16)
plt.yticks(range(4))
plt.title(str(plottn) + ' time-bins, ' + str(int(plotirf*1000)) + 'ps IRF')
plt.ylabel('F-value')
plt.xlabel('lifetime (ns)')

axins = ax.inset_axes([7.4, 0.15, 8, 1], transform=ax.transData)
axins.plot((-1, 16), (1,1), color='gray')
axins.plot(taus, np.squeeze(F_known[:,tn_idx_mc, irf_idx_mc]))
axins.plot(taus, np.squeeze(F_unknown[:,tn_idx_mc, irf_idx_mc]))
axins.plot(Tau0, kollner(Tau0, T, plottn)**(1/2), 'k--')
x1, x2, y1, y2 = -0.25, 3.75, 0.95, 1.45
axins.set_yticks(range(4))
axins.set_xticks((0,5,10,15))
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticklabels('')
axins.set_yticklabels('')
ax.indicate_inset_zoom(axins)
plt.legend(('measure', 'fit'))
plt.show
#plt.savefig('F_FisherMC_KnownUnknownGauss_' + str(plottn) + '.pdf')

In [ ]:
# Fig 3g
# Loss for measured and fitted IRF

# Determine index for parameters to plot
plottn = np.array((16, 256))
tn_idx_mc = np.in1d(tns, plottn)
tn_idx_mc = np.where(tn_idx_mc==True)[0]

# Compute loss
lossdata = F_unknown - F_known
L = np.zeros((tns.size, irfSigs.size))

for b in range(tns.size):
    for s in range(irfSigs.size):
        L[b,s] = metrics.auc(taus[1:], lossdata[1:,b,s])

# Generate plots
custom_cycler = (cycler(color=['#e377c2', '#bcbd22']))
fig, ax = plt.subplots(figsize=[2, 2])
ax.set_prop_cycle(custom_cycler)
ax.plot(irfSigs, np.transpose(L)[:,tn_idx_mc])
plt.ylabel('loss (ns)')
plt.xlabel('IRF (ns)')
plt.show
#plt.legend(tns[tn_idx_mc])
plt.title('Fit IRF - Meas. IRF')
#plt.savefig('KnownUnknownLoss.pdf')

### Figure 4: Separability and Resolving Power

In [ ]:
# Figure 4a 
# Separability
# Short vs Long Lifetimes

# Calculate Separability
tau1 = 0.5
tau2 = 0.55
tbin = 256
phots = 2000

tau1_idx = (np.abs(Tau0 - tau1)).argmin()
tau2_idx = (np.abs(Tau0 - tau2)).argmin()
bin_idx = (np.abs(Bin0 - tbin)).argmin()

tau1_F = fisherdata[tau1_idx, bin_idx, : ]**(1/2)
tau2_F = fisherdata[tau2_idx, bin_idx, : ]**(1/2)

tau1_sig = tau1_F*Tau0[tau1_idx]/np.sqrt(phots)
tau2_sig = tau2_F*Tau0[tau2_idx]/np.sqrt(phots)

S_lo = np.abs(tau1-tau2)/np.sqrt(tau1_sig**2 + tau2_sig**2)

tau1 = 2
tau2 = 2.2
tbin = 256
phots = 2000

tau1_idx = (np.abs(Tau0 - tau1)).argmin()
tau2_idx = (np.abs(Tau0 - tau2)).argmin()
bin_idx = (np.abs(Bin0 - tbin)).argmin()

tau1_F = fisherdata[tau1_idx, bin_idx, : ]**(1/2)
tau2_F = fisherdata[tau2_idx, bin_idx, : ]**(1/2)

tau1_sig = tau1_F*Tau0[tau1_idx]/np.sqrt(phots)
tau2_sig = tau2_F*Tau0[tau2_idx]/np.sqrt(phots)

S_hi = np.abs(tau1-tau2)/np.sqrt(tau1_sig**2 + tau2_sig**2)

# Generate plots
plt.figure(figsize=[2, 2])
plt.plot(Sig0, S_lo, marker='D', markersize=3)
plt.plot(Sig0, S_hi, marker='D', markersize=3)
plt.plot((min(Sig0), max(Sig0)), (2,2), 'k--')
plt.yticks(range(4))
plt.ylabel('separability')
plt.xlabel('IRF (ns)')
plt.legend(('short', 'long'))
plt.title(str(tbin) + ' time-bins')
plt.show
#plt.savefig("Separ_LowHi_marker.pdf")

# Estimate at S=2
print('S_lo below 2 at IRF of: ', np.interp(2, S_lo[::-1], Sig0[::-1]))
print('S_hi below 2 at IRF of: ', np.interp(2, S_hi[::-1], Sig0[::-1]))

In [ ]:
# Figure 4b
# Separability Plots
# Photon count needed for IRF of 0.1 and 1.0ns

# Calculate separability for short lifetime probe
tau1 = 0.5
tau2 = 0.55
irf1 = 0.1
irf2 = 1
tbin = 256
phots = np.arange(100,15000,300)

tau1_idx = (np.abs(Tau0 - tau1)).argmin()
tau2_idx = (np.abs(Tau0 - tau2)).argmin()
irf1_idx = (np.abs(Sig0 - irf1)).argmin()
irf2_idx = (np.abs(Sig0 - irf2)).argmin()
bin_idx = (np.abs(Bin0 - tbin)).argmin()

tau1irf1_F = fisherdata[tau1_idx, bin_idx, irf1_idx]**(1/2)
tau2irf1_F = fisherdata[tau2_idx, bin_idx, irf1_idx]**(1/2)
tau1irf2_F = fisherdata[tau1_idx, bin_idx, irf2_idx]**(1/2)
tau2irf2_F = fisherdata[tau2_idx, bin_idx, irf2_idx]**(1/2)

tau1irf1_sig = tau1irf1_F*Tau0[tau1_idx]/np.sqrt(phots)
tau2irf1_sig = tau2irf1_F*Tau0[tau2_idx]/np.sqrt(phots)
tau1irf2_sig = tau1irf2_F*Tau0[tau1_idx]/np.sqrt(phots)
tau2irf2_sig = tau2irf2_F*Tau0[tau2_idx]/np.sqrt(phots)

irf1_S = np.abs(tau1-tau2)/np.sqrt(tau1irf1_sig**2 + tau2irf1_sig**2)
irf2_S = np.abs(tau1-tau2)/np.sqrt(tau1irf2_sig**2 + tau2irf2_sig**2)

# Generate plots
custom_cycler = (cycler(color=['#1f77b4', '#d62728']))
fig, ax = plt.subplots(figsize=[2, 2])
ax.set_prop_cycle(custom_cycler)
ax.plot(phots, irf1_S)
ax.plot(phots, irf2_S)
ax.plot((min(phots), max(phots)), (2,2), 'k--')
plt.ylim(0,8)
plt.ylabel('separability')
plt.xlabel('photons collected')
plt.legend((irf1, irf2))
plt.title('Short: '+ "{:.2f}".format(tau1) +' - '+ "{:.2f}".format(tau2) +'ns')
plt.show
#plt.savefig("Separ_PhotonColl_LoTau.pdf")

In [ ]:
# Figure 4c
# Separability Plots
# Photon count needed for IRF of 0.1 and 1.0ns

# Calculate separability for long lifetime probes
tau1 = 2.0
tau2 = 2.2
irf1 = 0.1
irf2 = 1
tbin = 256
phots = np.arange(100,15000,300)

tau1_idx = (np.abs(Tau0 - tau1)).argmin()
tau2_idx = (np.abs(Tau0 - tau2)).argmin()
irf1_idx = (np.abs(Sig0 - irf1)).argmin()
irf2_idx = (np.abs(Sig0 - irf2)).argmin()
bin_idx = (np.abs(Bin0 - tbin)).argmin()

tau1irf1_F = fisherdata[tau1_idx, bin_idx, irf1_idx]**(1/2)
tau2irf1_F = fisherdata[tau2_idx, bin_idx, irf1_idx]**(1/2)
tau1irf2_F = fisherdata[tau1_idx, bin_idx, irf2_idx]**(1/2)
tau2irf2_F = fisherdata[tau2_idx, bin_idx, irf2_idx]**(1/2)

tau1irf1_sig = tau1irf1_F*Tau0[tau1_idx]/np.sqrt(phots)
tau2irf1_sig = tau2irf1_F*Tau0[tau2_idx]/np.sqrt(phots)
tau1irf2_sig = tau1irf2_F*Tau0[tau1_idx]/np.sqrt(phots)
tau2irf2_sig = tau2irf2_F*Tau0[tau2_idx]/np.sqrt(phots)

irf1_S = np.abs(tau1-tau2)/np.sqrt(tau1irf1_sig**2 + tau2irf1_sig**2)
irf2_S = np.abs(tau1-tau2)/np.sqrt(tau1irf2_sig**2 + tau2irf2_sig**2)

# Generate plots
custom_cycler = (cycler(color=['#1f77b4', '#d62728']))
fig, ax = plt.subplots(figsize=[2, 2])
ax.set_prop_cycle(custom_cycler)
ax.plot(phots, irf1_S)
ax.plot(phots, irf2_S)
ax.plot((min(phots), max(phots)), (2,2), 'k--')
plt.ylabel('separability')
plt.ylim(0,8)
plt.xlabel('photons collected')
plt.legend((irf1, irf2))
plt.title('Long: '+ "{:.2f}".format(tau1) +' - '+ "{:.2f}".format(tau2) +'ns')
plt.show
#plt.savefig("Separ_PhotonColl_HiTau.pdf")

In [ ]:
# Figure 4d
# Resolving power

# Calculate resolving power
tau1 = 0.5
tau2 = 2
tbin = 256
phots = 2000

tau1_idx = (np.abs(Tau0 - tau1)).argmin()
tau2_idx = (np.abs(Tau0 - tau2)).argmin()
bin_idx = (np.abs(Bin0 - tbin)).argmin()

tau1_F = fisherdata[tau1_idx, bin_idx, : ]**(1/2)
tau2_F = fisherdata[tau2_idx, bin_idx, : ]**(1/2)

tau1_R = np.sqrt(phots/(8*tau1_F**2))
tau2_R = np.sqrt(phots/(8*tau2_F**2))

# Generate plots
plt.figure(figsize=[2, 2])
plt.plot(Sig0, tau1_R, marker='D', markersize=3)
plt.plot(Sig0, tau2_R, marker='D', markersize=3)
plt.yticks(range(0, 16, 5))
plt.ylabel('resolving power')
plt.xlabel('IRF (ns)')
plt.title(str(tbin) + ' time-bins')
plt.legend(('short', 'long'))
plt.show
#plt.savefig("ResolvPow_LowHi_marker.pdf")

In [ ]:
# Figure 4e
# Plot of Resolving Power vs tau

# Determine index for parameters to plot
plotirf = np.array((0.1, 0.25, 0.5, 1, 2))
irf_idx = np.in1d(Sig0, plotirf)
irf_idx = np.where(irf_idx==True)[0]

irf1 = 2
irf1_idx = (np.abs(Sig0 - irf1)).argmin()

tbin = 256
bin_idx = (np.abs(Bin0 - tbin)).argmin()

# Calculate R from F
phots = 2000
Rval = 3
R = np.sqrt(phots/(8*fisherdata))
Rplot = R[:, bin_idx, irf_idx]

# Interpolate at Rval on irf1 line to get tau_R. Used to draw dashed line
R1 = R[:, bin_idx, irf1_idx]
tau_R=np.interp(Rval,R1[:np.argmax(R1)], Tau0[:np.argmax(R1)])

# Generate plots
plt.figure(figsize=[2, 2])
plt.plot(Tau0, Rplot)
plt.plot((0,tau_R), (Rval,Rval), 'k--')
plt.plot((tau_R,tau_R), (0,Rval), 'k--')
plt.yticks(range(0, 16, 5))
plt.ylabel('resolving power')
plt.xlabel('lifetime (ns)')
plt.title(str(tbin) + ' time-bins')
plt.legend(Sig0[irf_idx])
plt.show
#plt.savefig("ResolvPow_Tau_withInterp.pdf")

In [ ]:
# Figure 4f
# Min lifetime at IRF

# Determine index for parameters to plot
plottn = np.array((4, 16, 64, 256, 1024))
tn_idx = np.in1d(Bin0, plottn)
tn_idx = np.where(tn_idx==True)[0]

# Determine tau for Rval
tauR = np.zeros((Bin0.size, Sig0.size))

for b in range(Bin0.size):
    for s in range(Sig0.size):
        Rtemp = R[:,b,s]
        tauR[b,s] = np.interp(Rval, Rtemp[:np.argmax(Rtemp)], Tau0[:np.argmax(Rtemp)])

# Generate plots
custom_cycler = (cycler(color=['#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']))
fig, ax = plt.subplots(figsize=[2, 2])
ax.set_prop_cycle(custom_cycler)
plt.loglog(Sig0, np.transpose(tauR)[:,tn_idx], marker='D', markersize=3)
plt.ylabel('min. lifetime (ns)')
plt.xlabel('IRF (ns)')
#plt.legend(Bin0[tn_idx])
plt.title('Resolving power of ' + str(Rval))
plt.show
#plt.savefig("MinTau_IRF_marker_R6_Phot4000_nolegend.png")

### Misc

In [ ]:
#F value of 256bins, 100ps irf, tau = 0.050ns, R = 3

R_minTau = 3
plotirf = 0.1
plottn = 256
plottau = 0.02

irf_idx = np.in1d(Sig0, plotirf)
irf_idx = np.where(irf_idx==True)[0]

tn_idx = np.in1d(Bin0, plottn)
tn_idx = np.where(tn_idx==True)[0]

tau_idx = (np.abs(Tau0 - plottau)).argmin()

F_minTau = fisherdata[tau_idx, tn_idx, irf_idx]**(1/2)
p_minTau = F_minTau**(-2)
N_minTau = (8*R_minTau**2)/p_minTau
print("For IRF={}, Bins={}, and tau={}: \n p={}\n N={}\n".format(plotirf, plottn, plottau, p_minTau, N_minTau))

#Comparing detectors example

irf_idx = np.in1d(Sig0, 0.1)
irf_idx = np.where(irf_idx==True)[0]

tn_idx = np.in1d(Bin0, 256)
tn_idx = np.where(tn_idx==True)[0]

F_detector = fisherdata[:, tn_idx, irf_idx]**(1/2)
k_detector = kollner(Tau0, T, Bin0[tn_idx])**(1/2)
FisherDiracRatio = F_detector/k_detector[:,np.newaxis]

detector_idx = (np.abs(FisherDiracRatio - 1.01)).argmin()
print("IRF minimal effect at: tau={:.3f}ns".format(Tau0[detector_idx]))

detector_idx = (np.abs(FisherDiracRatio - 1.7)).argmin()
print("IRF greater than gain from collected photons at: tau={:.3f}ns".format(Tau0[detector_idx]))

In [ ]:
#Table 1: Minimum Resolvable Lifetime for R=3 and 2000 photons

phots = 2000
Rval = 3

plottn = np.array((4, 16, 64, 256, 1024))
tn_idx = np.in1d(Bin0, plottn)
tn_idx = np.where(tn_idx==True)[0]

plotirf = np.array((0.1, 0.25, 0.5, 1, 2))
irf_idx = np.in1d(Sig0, plotirf)
irf_idx = np.where(irf_idx==True)[0]

plottauR = tauR[tn_idx,:]
plottauR = plottauR[:,irf_idx]

df=pd.DataFrame(np.around(np.transpose(plottauR),2), columns=plottn)
df=pd.concat([pd.DataFrame({'IRF (ns)':plotirf}), df], axis=1)
df=df.set_index('IRF (ns)')
df

In [ ]:
# Table 2: Photons required for R=3

R_minTau = 3

plottn = 256
plottau = np.array((0.02, 0.05, 0.10, 0.25, 0.50))
plotirf = np.array((0.1, 0.25, 0.5, 1, 2))

tn_idx = np.in1d(Bin0, plottn)
tn_idx = np.where(tn_idx==True)[0]

tau_idx = np.zeros((plottau.shape))

irf_idx = np.in1d(Sig0, plotirf)
irf_idx = np.where(irf_idx==True)[0]

for idx in range(plottau.size):
    tau_idx[idx] = (np.abs(Tau0 - plottau[idx])).argmin()

tau_idx = tau_idx.astype(int)

p = 1/fisherdata
p = p[tau_idx,:,:]
p = p[:,tn_idx,:]
p = p[:,:,irf_idx]
p = np.squeeze(np.moveaxis(p, [0, 1, 2], [1, 2, 0]))
N = (8*R_minTau**2)/p

df2=pd.DataFrame(np.around(N.astype(int),-1), columns=plottau)
df2=pd.concat([pd.DataFrame({'IRF (ns)':plotirf}), df2], axis=1)
df2=df2.set_index('IRF (ns)')
df2